# W_GL_SEGMENT_DS ETL Process
### Oracle EBS FND_FLEX_VALUES to EDW W_GL_SEGMENT_DS (Staging)

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2624634 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Extract from FND_FLEX_VALUES with effective dating
CREATE OR REPLACE TEMP VIEW stg_flex_values AS
SELECT
  CAST(fvs.flex_value_set_id AS STRING) AS segment_lov_id,
  CASE 
    WHEN fvs.validation_type = 'I' THEN COALESCE(fv.flex_value, '__UNASSIGNED__')
    ELSE COALESCE(fv.flex_value || '~' || fv.parent_flex_value_low, '__UNASSIGNED__')
  END AS segment_val_code,
  CAST(fv.created_by AS STRING) AS created_by_id,
  CAST(fv.last_updated_by AS STRING) AS changed_by_id,
  fv.creation_date AS created_on_dt,
  fv.last_update_date AS changed_on_dt,
  fvs.last_update_date AS aux1_changed_on_dt,
  to_date('1899-01-01', 'yyyy-MM-dd') AS src_eff_from_dt,
  to_date('3714-01-01', 'yyyy-MM-dd') AS src_eff_to_dt,
  'N' AS delete_flg,
  p.DATASOURCE_NUM_ID,
  CAST(fvs.flex_value_set_id AS STRING) || '~' || fv.flex_value AS integration_id,
  p.TENANT_ID, '0' AS x_custom
FROM workspace.oracle_ebs.FND_FLEX_VALUES fv
INNER JOIN workspace.oracle_ebs.FND_FLEX_VALUE_SETS fvs
  ON fv.flex_value_set_id = fvs.flex_value_set_id
CROSS JOIN etl_params p
WHERE fvs.validation_type IN ('I', 'D')
  AND (fv.last_update_date > to_timestamp(p.LAST_EXTRACT_DATE, 'yyyy-MM-dd HH:mm:ss')
    OR fvs.last_update_date > to_timestamp(p.LAST_EXTRACT_DATE, 'yyyy-MM-dd HH:mm:ss'));

In [ ]:
%sql
-- 3) INSERT: Load data into target table W_GL_SEGMENT_DS
INSERT INTO workspace.oracle_edw.w_gl_segment_ds
SELECT 
  segment_lov_id, segment_val_code, created_by_id, changed_by_id,
  created_on_dt, changed_on_dt, aux1_changed_on_dt,
  src_eff_from_dt, src_eff_to_dt, delete_flg,
  datasource_num_id, integration_id, tenant_id, x_custom
FROM stg_flex_values;

In [ ]:
%sql
-- 4) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION' AS package_name,
    'W_GL_SEGMENT_DS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 5) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION', 'W_GL_SEGMENT_DS', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_flex_values;
DROP VIEW IF EXISTS etl_params;